## MLP
- 사용 데이터: QMNIST
- 데이터 크기: (60000, 28, 28) 

이 튜토리얼은 QMNIST 숫자를 분류하기 위해 다층 퍼셉트론(Multi-Layer Perceptron)을 훈련합니다. <br>
이 모델을 QMNIST 테스트 세트에서 96% 정확도를 달성합니다.

### 라이브러리 임포트하기
- torch 라이브러리 install을 위해 참고할 사이트: https://pytorch.org/get-started/locally/
- numpy 라이브러리 install을 위해 사용한 코드: `pip install numpy`

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
import random

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True

## 데이터 로드
- 데이터 불러오기(pytorch에 내장된 데이터, 처음 실행 시 지정 폴더에 다운됨)
- 데이터 구성을, 학습/테스트 형태로 구분
- MNIST 데이터와 같이 비전 모델 개발에 자주 사용되는 데이터는 학습, 테스트 데이터 구성 이미 정의


In [3]:
transform = transforms.Compose([
    # transforms.Resize() : 이미지 사이즈 변경
    transforms.ToTensor(),                          # 이미지를 텐서로 변환(다차원 배열구조)
    transforms.Normalize((0.1307,), (0.3081))       # 데이터 평균, 표준편차 값 활용해 정규화(ex. 0~255 -> -1~1)
])

train_dataset = datasets.QMNIST('/data', train=True,  transform=transform, download=True)    # 지정 폴더 = 코드 실행 위치 내 data라는 이름의 폴더
test_dataset = datasets.QMNIST('/data', train=False, transform=transform, download=True)                    # 학습, 테스트 데이터 구분해 다운

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)       # batch_size: 모델에 한번에 입력할 sample 수
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)      # shuffle: 데이터 랜덤하

100%|██████████| 9742279/9742279 [01:09<00:00, 140523.68it/s] 


Extracting /data\QMNIST\raw\qmnist-test-images-idx3-ubyte.gz to /data\QMNIST\raw


100%|██████████| 526800/526800 [00:00<00:00, 10160294.97it/s]

Extracting /data\QMNIST\raw\qmnist-test-labels-idx2-int.gz to /data\QMNIST\raw

## 모델 설계
위 모델은 입력층, 은닉층, 출력층으로 이루어져 있으며, ReLU 활성화 함수를 사용한 세 개의 완전 연결 층으로 구성되어 있습니다. 

### MLP 구현
- QMNIST dataset(28*28, 60000) -> Input layer(784=28x28) -> Hidden layer(512) -> output layer(256) -> Hidden layer(256) -> output layer(10)
- 출력 값은 0~9(10가지) 중 하나에 해당
- torch.nn.Module을 상속받아 모델 작성

### 모델 객체 생성
- 손실함수(criterion): Negative Log Likelihood Loss
- 최적화: 확률적 경사 하강법 사용
- lr: learning rate 크기가 클수로 빠른 속도 학습, 정답 착기 어려움(작으면 속도 느림)
- momentum: 로컬 미니마에 빠질 위험을 줄여주기 우한 요소(0~1, 값이 클수록 영향력 큼)

### 모델 학습 메소드 구현
- device: GPU 사용 여부



In [21]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
    
model = MLP()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(model, device, train_loader, optimizer, epoch):   
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):   # 전체 데이터를 barch size만큼 읽어오는 과정 반복
        data, target = data.to(device), target.to(device)       # GPU 사용시 CPU -> GPU
        optimizer.zero_grad()       # Gradient 초기화, 매 학습시마다 초기화 후 backward
        output = model(data)        # 데이터를 MLP에 통과치켜 output 획득
        loss = criterion(output, target)        # output 벡터와 정답을 이용해 손실값 계산
        loss.backward()             # 계산된 손실값으로 backward 진행(변화량 계산, 미분 계산)
        optimizer.step()            # 파라미터 업데이트
        if batch_idx % 100 == 0:    # 결과값 10번 반복마다 print
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}({100.*batch_idx/len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')




def test(model, device, test_loader, epoch):
    model.eval()        # 모델 평가 시, eval() 적용해 평가모드 진입(학습시 필요 연산 비활성화)
    test_loss = 0       # 테스트 데이터에 대한 누적 손실값 계산을 위한 변수
    correct = 0         # 테스트 데이터들 중, 정답을 맞춘 데이터의 수를 세기 위한 변수
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()       # 손실함수를 통한 손실값 계산
            pred = output.argmax(dim=1, keepdim=True)           # 0~9중 가장 높은 확률에 해당하는 값
            correct += pred.eq(target.view_as(pred)).sum().item()   # 예측치와 실제 닶이 같은지 비교

    test_loss /= len(test_loader.dataset)           # 전체 데이터 수로 누적값을 나눠, 평균 손실값 계산
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy:{correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n', end='\r')
    # 전체 데이터에 대해 정답을 맞춘 비율(정확도) 출력


# 모델 저장 및 불러오기 함수


In [22]:
def save_model(model, epoch):
    torch.save(model.state_dict(), 'models/mnist_mlp_model{}.pth'.format(epoch))

def load_model(model, model_path):
    model.load_state_dict(torch.load(model_path))

## 학습과 저장
다음 코드 결과 .pth로 저장 / models 파일 생성 해야 함

In [23]:
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    epochs = 5

    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader, epoch)
        save_model(model, epoch)

Train Epoch: 1 [0/60000(0%)]	Loss: 2.324884
Train Epoch: 1 [6400/60000(11%)]	Loss: 1.092968
Train Epoch: 1 [12800/60000(21%)]	Loss: 0.610828
Train Epoch: 1 [19200/60000(32%)]	Loss: 0.363984
Train Epoch: 1 [25600/60000(43%)]	Loss: 0.268792
Train Epoch: 1 [32000/60000(53%)]	Loss: 0.339016
Train Epoch: 1 [38400/60000(64%)]	Loss: 0.370975
Train Epoch: 1 [44800/60000(75%)]	Loss: 0.440110
Train Epoch: 1 [51200/60000(85%)]	Loss: 0.343822
Train Epoch: 1 [57600/60000(96%)]	Loss: 0.164796

Test set: Average loss: 0.0003, Accuracy:55214/60000 (92%)
Train Epoch: 2 [0/60000(0%)]	Loss: 0.283648
Train Epoch: 2 [6400/60000(11%)]	Loss: 0.287534
Train Epoch: 2 [12800/60000(21%)]	Loss: 0.358240
Train Epoch: 2 [19200/60000(32%)]	Loss: 0.373477
Train Epoch: 2 [25600/60000(43%)]	Loss: 0.156273
Train Epoch: 2 [32000/60000(53%)]	Loss: 0.239027
Train Epoch: 2 [38400/60000(64%)]	Loss: 0.176559
Train Epoch: 2 [44800/60000(75%)]	Loss: 0.140408
Train Epoch: 2 [51200/60000(85%)]	Loss: 0.310318
Train Epoch: 2 [57600

# 개선 방안 및 시도

### 1. MLP 층 늘리기
```
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)           # 층을 하나 더 추가해서 정확도가 증가하는지 측정
```
> - 기존에 fc2까지 작성 후 정확도: 55112/60000 (92%) <br>
> - 변경 후 fc3까지 작성 후 정확도: 55352/60000 (92%)
>> 결과: 약 200개 정도의 데이터를 더 맞춰지만, 정확도 퍼센트에 대한 큰 변화는 없었다


### 2. 출력함수, 손실함수 변화
- `return F.log_softmax(x, dim=1)`
- `criterion = nn.NLLLoss()`

> 기존 softmax, crossentropy를 사용한 정확도: 55352/60000 (92%)
> 변경 후 log_softmax, NLLLoss를 사용한 정확도: 57872/60000 (96%)
>> 결과: 약 2500개를 더 예측하며 정확도 퍼센트를 4% 더 올리는 유의미한 결과를 얻었다


#### - LogSoftmax 사용
 LogSoftmax는 주어진 차원에서 각 클래스에 대한 확률의 로그를 계산한다. <br>
 이 함수는 확률을 로그 공간에서 다루어 안정성을 높이고, 손실함수에 사용한 Negative Log Likelihood Loss를 위해 적용된다고 한다.

#### - NLLLoss 사용
CrossEntropyLoss는 내부적으로 LogSoftmax를 수행한다고 한다, 모델의 출력에 LogSoftmax를 사용하면서 NLLLoss를 사용해보았다.